In [62]:
class A:
    def __init__(self):
        self.x = 3
    def tt(self):
        return 1
    def ss(self):
        return self.x

In [63]:
dir(A)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'ss',
 'tt']

In [64]:
a = A()

In [65]:
dir(a)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'ss',
 'tt',
 'x']

In [25]:
a.ss()

3

**아래에서 확실하게 인스턴스 변수가 인스턴스 메서드 실행 시점에 생성됨을 확인한다**

In [66]:
class B:
    def x(self):
        self.a = 2

In [67]:
b = B()

In [68]:
vars(b)

{}

In [69]:
b.x()

In [70]:
vars(b)

{'a': 2}

In [71]:
#동적으로 아무거나 생성할 수도 있다.
b.t = 1

In [72]:
vars(b)

{'a': 2, 't': 1}

어떤 방식을 사용하면 이 동적인 파이썬에서 실수를 가장 줄일 수 있을까? 미리 사용할 애를 처음부터 \_\_init\_\_에 만들어놓고, 동적으로 생성하지는 않는것이 좋다.

컴포지션이란 나의 인스턴스 변수에 다른 컴포넌트의 변수를 저장하는것이다. 상속도 컴포지션으로 구현할 수 있다.

# 02 상속을 컴포지션으로 구현

In [77]:
class A:
    x = 1
    def t(self):
        return 1

In [78]:
class B(A):
    x = 2

In [79]:
class C:      #저질 방식
    x = 2
    def t(self):
        return A.t(self)

In [80]:
c = C()

In [82]:
c.t()

1

In [83]:
vars(c)

{}

In [44]:
class D:
    def __init__(self):
#         self.x = A.x
        self.tt = A()
    
    def t(self):
        return self.tt.t()

In [45]:
d = D()

In [47]:
d.t()

1

In [52]:
class D:
    def __init__(self):
#         self.x = A.x
        self.tt = A()
    
    def __getattr__(self, x):     #예외처리 : attribute예외가 나오면 얘가 실행된다. 즉 *.x()가 없으면 에러가 난다.
        return getattr(self.tt, x)

In [53]:
dd = D()

In [54]:
dd.x

1

getattr은 뭐다?

In [60]:
#a = __abs__
a = input()

__abs__


In [61]:
getattr(-1, a)()

1

In [86]:
import tensorflow as tf

In [97]:
class D:
    def call(self):
        raise NotImplementedError
    def __call__(self):
        self.call()

In [102]:
class E(D):
    def call(self):
        print('call')

In [103]:
e = E()

In [104]:
#상속할 때 무조건 구현하도록, call()에서 raise NotImplementedError
e.call()

call


In [105]:
e()

call


# 03 Lazy Evaluation

파이토치는 데이터 확인이 힘든 레이지 방식 대신 eager를 사용해서 값을 확인할 수 있게 했기 때문에 유행하였다.<br>
텐서플로우도 eager tensor를 쓰며, eager가 더 편하고, 이것이 기본값이기 때문에 lazy기법에 대해서 따로 장단점을 알아야 한다.

파이썬교재 02 26페이지

# 04 Abstract

파이썬의 추상화는 상속의 문법을 가진다.<br>
파이썬의 추상화는 metaclass를 이용한다.

In [107]:
from sklearn.naive_bayes import ABCMeta #Abstract Base Class

In [108]:
from sklearn.naive_bayes import ( BernoulliNB, CategoricalNB, ComplementNB, GaussianNB, MultinomialNB )

파이썬관점에서 공통된점(가장 좋은 점)을 뽑아서 부모객체로 만드는 것이 추상화이다.<br>
그 뛰어난 공통점 아래에서 특수화해야 하는 부분을 구현을 해야만 하도록 하는 것이 효율적이다.<br>
이러한 변형에서 사용하는 것이 ABCMeta<br>
모델은 여러가지가 있지만, 내가 만족할 수 있는 모델은 없다. 여러개 지원해주면 가능성이 있고 여기서 뽑아내서 내 모델 만들기 위해 ABCMeta 사용한다.

In [109]:
A, B, C

(__main__.A, __main__.B, __main__.C)

In [115]:
class D:
    def c(self): #공통기능
        pass
    def ee(self): #차별적인 기능
        raise NotImplementedError

In [116]:
class A(D):
    pass

In [117]:
a = A()

In [123]:
a.ee()

NotImplementedError: 

In [125]:
from collections.abc import Sequence

abstract base class

In [129]:
from collections.abc import Sequence 

class ExpandingSequence(Sequence):
    def __init__(self, it):
        self.it = it
        self._cache = [] 
    def __getitem__(self, index):
        while len(self._cache) <= index:
            self._cache.append(next(self.it))
        return self._cache[index] 
    def __len__(self):
        return len(self._cache)
    
#getitem : 인덱싱을 위해서
#len : -로 접근할 때 필요함.

In [134]:
import inspect

In [143]:
print(inspect.getsource(tf.keras.utils.Sequence))
# 텐서플로우 상속을 안받았네. 가능하다? 가능하다. 상속안받아도, 덕타이핑을 통해 가능하다.

@keras_export("keras.utils.Sequence")
class Sequence:
    """Base object for fitting to a sequence of data, such as a dataset.

    Every `Sequence` must implement the `__getitem__` and the `__len__` methods.
    If you want to modify your dataset between epochs, you may implement
    `on_epoch_end`. The method `__getitem__` should return a complete batch.

    Notes:

    `Sequence` is a safer way to do multiprocessing. This structure guarantees
    that the network will only train once on each sample per epoch, which is not
    the case with generators.

    Examples:

    ```python
    from skimage.io import imread
    from skimage.transform import resize
    import numpy as np
    import math

    # Here, `x_set` is list of path to the images
    # and `y_set` are the associated classes.

    class CIFAR10Sequence(tf.keras.utils.Sequence):

        def __init__(self, x_set, y_set, batch_size):
            self.x, self.y = x_set, y_set
            self.batch_size = batch_size

     

In [140]:
from torch.utils.data import DataLoader, Dataset

In [141]:
print(inspect.getsource(Dataset))

class Dataset(Generic[T_co]):
    r"""An abstract class representing a :class:`Dataset`.

    All datasets that represent a map from keys to data samples should subclass
    it. All subclasses should overwrite :meth:`__getitem__`, supporting fetching a
    data sample for a given key. Subclasses could also optionally overwrite
    :meth:`__len__`, which is expected to return the size of the dataset by many
    :class:`~torch.utils.data.Sampler` implementations and the default options
    of :class:`~torch.utils.data.DataLoader`.

    .. note::
      :class:`~torch.utils.data.DataLoader` by default constructs a index
      sampler that yields integral indices.  To make it work with a map-style
      dataset with non-integral indices/keys, a custom sampler must be provided.
    """

    def __getitem__(self, index) -> T_co:
        raise NotImplementedError

    def __add__(self, other: 'Dataset[T_co]') -> 'ConcatDataset[T_co]':
        return ConcatDataset([self, other])

    # No `def 

In [170]:
#얘도 상속을 안받아도 된다.
from collections.abc import Sequence 

class ExpandingSequence():
    def __init__(self, it):
        self.it = it
        self._cache = [] 
    def __getitem__(self, index):
        while len(self._cache) <= index:
            self._cache.append(next(self.it))
        return self._cache[index] 
    def __len__(self):
        return len(self._cache)

In [171]:
e = ExpandingSequence(iter([1,2,3,4]))

In [172]:
e[0]

1

In [173]:
e[-1] #4여야 되는데. 어쨌든 덕타이핑 해서 작동만 하면 엉터리 동작이여도 그냥 막 한다.

1

In [178]:
#얘도 상속을 안받아도 된다.
from collections.abc import Sequence 

class ExpandingSequence(Sequence):
    def __init__(self, it):
        self.it = it
        self._cache = [] 
    def __getitem__(self, index):
        while len(self._cache) <= index:
            self._cache.append(next(self.it))
        return self._cache[index] 
#     def __len__(self):
#         return len(self._cache)

In [182]:
e = ExpandingSequence(iter([1,2,3,4]))
#추상화되어있기 때문에, 이거 구현해야해요 하고, 전체적인 구조를 짠 사람이 짜놓았기 때문에 __len__이 없을 때 경고를 보인다.

TypeError: Can't instantiate abstract class ExpandingSequence with abstract method __len__

In [183]:
e[1]

2

In [189]:
from abc import abstractmethod, ABC

In [192]:
class D(ABC):
    @abstractmethod
    def x(self):
        pass

In [193]:
class E(D):
    pass

In [194]:
e = E()

TypeError: Can't instantiate abstract class E with abstract method x

텐서플로우는 상속 안하는데, 이럴 때 @abstractmethod는 가이드만 해주고, 상속해야 에러가 발생한다.

In [195]:
print(inspect.getsource(Dataset))

class Dataset(Generic[T_co]):
    r"""An abstract class representing a :class:`Dataset`.

    All datasets that represent a map from keys to data samples should subclass
    it. All subclasses should overwrite :meth:`__getitem__`, supporting fetching a
    data sample for a given key. Subclasses could also optionally overwrite
    :meth:`__len__`, which is expected to return the size of the dataset by many
    :class:`~torch.utils.data.Sampler` implementations and the default options
    of :class:`~torch.utils.data.DataLoader`.

    .. note::
      :class:`~torch.utils.data.DataLoader` by default constructs a index
      sampler that yields integral indices.  To make it work with a map-style
      dataset with non-integral indices/keys, a custom sampler must be provided.
    """

    def __getitem__(self, index) -> T_co:
        raise NotImplementedError

    def __add__(self, other: 'Dataset[T_co]') -> 'ConcatDataset[T_co]':
        return ConcatDataset([self, other])

    # No `def 

텐서플로우, 파이토치 모두 덕타이핑 방식이다. 그런데 에러 대신 가이드 방식이다. 왜냐면 전문가들이니까. 근데 ABC상속을 통해서 강제시킬수도 있기는 하다.

## 04.01 MetaClass

In [198]:
from abc import ABCMeta, abstractmethod

**메타클래스?**<br>
클래스의 행동을 결정해주는 애. 얘를 이용하는게 가장 세련되다. 그래서 사이킷런에도 메타클래스를 사용한다.

In [202]:
class E(metaclass=ABCMeta):
    @abstractmethod
    def x(self):
        pass
#메타클래스는 상속하고 기능을 덧붙일 수 있어 좋음

In [203]:
class F(E):
    pass

In [204]:
f = F()

TypeError: Can't instantiate abstract class F with abstract method x

# 05 Map, Iterable

map에 보면 *iterable이 있다. 즉 iterable을 여러개 받는다.

파이토치에는 map style과 iteration style 으로 나누어놨다.<br>
tensorflow에서도 이터러블과 맵 나누어놓았다.<br>
pandas도 맵으로 끝난다.<br>
맵이 더 우아하다.

In [221]:
tips = sns.load_dataset('tips')

In [222]:
tips['total_bill']

0      16.99
1      10.34
2      21.01
3      23.68
4      24.59
       ...  
239    29.03
240    27.18
241    22.67
242    17.82
243    18.78
Name: total_bill, Length: 244, dtype: float64

In [223]:
dir(tips['total_bill'])

['T',
 '_AXIS_LEN',
 '_AXIS_ORDERS',
 '_AXIS_TO_AXIS_NUMBER',
 '_HANDLED_TYPES',
 '__abs__',
 '__add__',
 '__and__',
 '__annotations__',
 '__array__',
 '__array_priority__',
 '__array_ufunc__',
 '__array_wrap__',
 '__bool__',
 '__class__',
 '__contains__',
 '__copy__',
 '__deepcopy__',
 '__delattr__',
 '__delitem__',
 '__dict__',
 '__dir__',
 '__divmod__',
 '__doc__',
 '__eq__',
 '__finalize__',
 '__float__',
 '__floordiv__',
 '__format__',
 '__ge__',
 '__getattr__',
 '__getattribute__',
 '__getitem__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__iadd__',
 '__iand__',
 '__ifloordiv__',
 '__imod__',
 '__imul__',
 '__init__',
 '__init_subclass__',
 '__int__',
 '__invert__',
 '__ior__',
 '__ipow__',
 '__isub__',
 '__iter__',
 '__itruediv__',
 '__ixor__',
 '__le__',
 '__len__',
 '__long__',
 '__lt__',
 '__matmul__',
 '__mod__',
 '__module__',
 '__mul__',
 '__ne__',
 '__neg__',
 '__new__',
 '__nonzero__',
 '__or__',
 '__pos__',
 '__pow__',
 '__radd__',
 '__rand__',
 '__rdivmod__',
 '__redu

In [224]:
temp = []
for i in tips['total_bill']: #accumulation
    temp.append(i*10)

In [225]:
#또는
b = iter(tips['total_bill'])

In [226]:
next(b)
#해서 하나씩 받아와서 해도 된다.

16.99

In [228]:
### 함수형 패러다임은 what에 집중해라
b = tips['total_bill'].map(lambda x: x*10)

In [229]:
b

0      169.9
1      103.4
2      210.1
3      236.8
4      245.9
       ...  
239    290.3
240    271.8
241    226.7
242    178.2
243    187.8
Name: total_bill, Length: 244, dtype: float64

In [231]:
tips['sex']

0      Female
1        Male
2        Male
3        Male
4      Female
        ...  
239      Male
240    Female
241      Male
242      Male
243    Female
Name: sex, Length: 244, dtype: category
Categories (2, object): ['Male', 'Female']

In [233]:
#남여를 바꾸고 싶다.
tips.sex.map({'Female':'F', 'Male':'M'})

0      F
1      M
2      M
3      M
4      F
      ..
239    M
240    F
241    M
242    M
243    F
Name: sex, Length: 244, dtype: category
Categories (2, object): ['M', 'F']

In [236]:
b = map(lambda x:x+10, [1,2,3,4,5])
b

In [237]:
next(b)

11

In [238]:
dir(b)
#로직에 집중할 수 있다.
#python은 맵도 iterable 기반이다.

['__class__',
 '__delattr__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__lt__',
 '__ne__',
 '__new__',
 '__next__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__']

In [242]:
b = map(lambda x,Y:x+Y, [1,2,3,4,5], [1,2,3,4,5])

In [243]:
next(b)

2

In [246]:
a = zip([1,2,3,4],[3,4,5])
#optimizer를 적용하기 위해서 여껑야하기 때문에 zip을 무조건 알아야 한다.
#zip에 문제점이 하나가 있는데, 크기가 다를 경우에, 잘라버린다.

In [250]:
next(a) #iter 3times

StopIteration: 

In [252]:
#따라서 긴거 기준으로 사용할 땐
from itertools import zip_longest

In [253]:
a = zip_longest([1,2,3,4],[3,4,5])

In [258]:
next(a) #iter 4times

StopIteration: 

In [259]:
from functools import reduce

In [261]:
reduce(lambda x,y: x+y, [1,2,3,4,5]) #iterable이 *도 없고, s도 없으니 한개만 쓴다. 왜? 연쇄적으로 줄인다.
#reduce의 반대되는 accumulation

15

In [263]:
from itertools import accumulate

In [264]:
b = accumulate([1,2,3,4,5], lambda x, y:x+y)

In [265]:
list(b)

[1, 3, 6, 10, 15]

# 06 Itertools

In [266]:
from itertools import accumulate, chain, cycle, groupby

In [267]:
b = cycle([1,2,3,4])

In [288]:
next(b) #iter infinite by while

3

In [289]:
b = chain([1,2,3,4],[3,4,5])

In [297]:
next(b) #iter seven times

StopIteration: 

파이토치에서 Dataset을 Dataloader을 써서 모델에 Feed 해준다.<br>
순서가 중요하면 iterable로 만든다.<br>
데이터가 크면 iterator로 만든다.

파이토치에서 제공하는 Dataset을 상속 받으면, 형태에 안맞으면 에러나게 추상화시켜놨는데,<br>
Sequnce는 iterable(\_\_iterable\_\_) 이었다. 